In [18]:
%pip install selenium webdriver-manager pandas


UnboundLocalError: cannot access local variable 'child' where it is not associated with a value

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import openai


In [ ]:
# Step 2: Set up the Selenium WebDriver with efficiency improvements
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode for efficiency
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
# Step 2: Set up the Selenium WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
# Step 3: Load the URLs from CSV
urls_df = pd.read_csv('/Users/miguelemb/Downloads/Projects/Grant scrapper/grant_db_csv/Business Incubator Grants (1).csv')
clean_urls = [url.split('"')[1] for url in urls_df['OPPORTUNITY NUMBER'].tolist() if isinstance(url, str)]



In [ ]:

all_grant_info = []

In [ ]:
# Define the paths to the different sections
table_paths = {
    "General Information": "/html/body/div[1]/div[4]/div/div/div/div[3]/div/div/div/div[1]/div/div[1]/table",
    "Eligibility Information": "/html/body/div[1]/div[4]/div/div/div/div[3]/div/div/div/div[2]/div/table",
    "Additional Information": "/html/body/div[1]/div[4]/div/div/div/div[3]/div/div/div/div[3]/div/table"
}

In [ ]:
for idx, url in enumerate(tqdm(clean_urls, desc="Processing URLs")):
    driver.get(url)
    time.sleep(0.5)  # Reduced wait time for faster processing

    grant_info = urls_df.iloc[idx].to_dict()  # Start with the original information from the CSV
    grant_info["URL"] = url
    
    # Loop through each table path and extract information
    for section, xpath in table_paths.items():
        try:
            table = driver.find_element(By.XPATH, xpath)
            rows = table.find_elements(By.TAG_NAME, "tr")
            section_info = {}
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    section_info[key] = value
            grant_info[section] = section_info
        except Exception as e:
            grant_info[section] = f"Error extracting {section}: {str(e)}"
    
    all_grant_info.append(grant_info)

# Step 5: Close the browser session
driver.quit()

Processing URLs: 100%|██████████| 510/510 [08:32<00:00,  1.01s/it]


In [ ]:
# Step 6: Convert to DataFrame and save
all_grants_df = pd.DataFrame(all_grant_info)


In [ ]:
all_grants_df.to_csv('all_grants.csv', index=False)